In [1]:
#importing all the required libraries and modules
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from pandas.tseries.holiday import USFederalHolidayCalendar
import scipy.stats
import matplotlib.dates as mdates
from google.cloud import bigquery

In [2]:
#connecting notebook to gcloud bigquery to query data.
cred_file = r"C:\Users\V0V01S7\AppData\Roaming\gcloud\legacy_credentials\venkatasaiyashaswi.v@walmart.com\adc.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=cred_file
def query_bq(QUERY):
    client = bigquery.Client(project='wmt-edw-sandbox')
    query_job = client.query(QUERY) # API request
    rows = query_job.result() # Waits for query to finish
    df = rows.to_dataframe()
    return df

In [3]:
code = """select A.DRVR_USER_ID, A.MARKET_NM,
B.ACTV_START_TS, B.ACTL_ACTV_END_TS, B.ACTV_DUR_MIN_QTY, B.ACTV_DT, 
from `wmt-edw-prod.WW_GEC_VM.DRVR` A inner join `wmt-edw-prod.WW_GEC_VM.DRVR_APPLN_ACTV` B on A.DRVR_USER_ID = B.DRVR_USER_ID
where A.MARKET_NM in ('Walmart San Jose North','Walmart San Jose South','Walmart Orlando','Walmart Virginia Beach', 'Walmart Tulsa','Walmart South Minneapolis') AND B.ACTV_NM = "ONLINE"
"""
df = query_bq(code)
df["ACTV_START_TS"] = pd.to_datetime(df["ACTV_START_TS"])
df["ACTL_ACTV_END_TS"] = pd.to_datetime(df["ACTL_ACTV_END_TS"])
#df["ACTV_START_TS"] = df["ACTV_START_TS"] + pd.Timedelta(hours=2) #converting pacific time to eastern time (orlando market)
#df["ACTL_ACTV_END_TS"] = df["ACTL_ACTV_END_TS"] + pd.Timedelta(hours=2)
df.head()

,DRVR_USER_ID,MARKET_NM,ACTV_START_TS,ACTL_ACTV_END_TS,ACTV_DUR_MIN_QTY,ACTV_DT
0,julieannpruitt@icloud.com,Walmart Tulsa,2021-03-17 14:15:05.216,2021-03-17 14:25:15.469,10.170000000,2021-03-17
1,busmom1805@gmail.com,Walmart Tulsa,2021-03-17 05:40:46.530,2021-03-17 07:56:37.586,135.850000000,2021-03-17
2,renemora91@hotmail.com,Walmart Tulsa,2021-03-17 05:05:34.162,2021-03-17 06:26:25.224,80.850000000,2021-03-17
3,renemora91@hotmail.com,Walmart Tulsa,2021-03-17 10:39:49.862,2021-03-17 13:30:00.000,170.170000000,2021-03-17
4,renemora91@hotmail.com,Walmart Tulsa,2021-03-17 07:57:35.398,2021-03-17 09:30:00.000,92.400000000,2021-03-17


In [4]:
df[df["DRVR_USER_ID"].str.contains("faisal")]

,DRVR_USER_ID,MARKET_NM,ACTV_START_TS,ACTL_ACTV_END_TS,ACTV_DUR_MIN_QTY,ACTV_DT
706,uzma4faisal@yahoo.com,Walmart Tulsa,2021-03-30 14:43:16.153,2021-03-30 14:53:32.684,10.270000000,2021-03-30
707,uzma4faisal@yahoo.com,Walmart Tulsa,2021-03-30 13:14:59.846,2021-03-30 14:40:29.186,85.480000000,2021-03-30
708,uzma4faisal@yahoo.com,Walmart Tulsa,2021-03-30 14:53:44.125,2021-03-30 16:00:00.000,66.250000000,2021-03-30
709,uzma4faisal@yahoo.com,Walmart Tulsa,2021-03-30 17:02:52.606,2021-03-30 18:00:00.000,57.120000000,2021-03-30
980,uzma4faisal@yahoo.com,Walmart Tulsa,2021-04-02 15:01:22.563,2021-04-02 18:00:00.000,178.620000000,2021-04-02
...,...,...,...,...,...,...
227725,uzma4faisal@yahoo.com,Walmart Tulsa,2022-06-03 17:39:25.481,2022-06-03 17:41:04.454,1.650000000,2022-06-03
227726,uzma4faisal@yahoo.com,Walmart Tulsa,2022-06-03 17:43:19.485,2022-06-03 17:44:21.064,1.030000000,2022-06-03
227727,uzma4faisal@yahoo.com,Walmart Tulsa,2022-06-03 17:44:21.064,2022-06-03 20:00:00.000,135.650000000,2022-06-03
228650,uzma4faisal@yahoo.com,Walmart Tulsa,2022-07-18 20:41:32.326,2022-07-11 16:30:00.000,-10331.530000000,2022-07-18


In [5]:
df = df[df["DRVR_USER_ID"].str.contains("walmart|faisal") == False]
len(df)

1270303

In [6]:
df = df[df["ACTV_DUR_MIN_QTY"] > 0 ]
len(df)

1263190

In [7]:
df = df.drop_duplicates()
len(df)

1091364

In [32]:
df.head()

,DRVR_USER_ID,MARKET_NM,ACTV_START_TS,ACTL_ACTV_END_TS,ACTV_DUR_MIN_QTY,ACTV_DT
0,julieannpruitt@icloud.com,Walmart Tulsa,2021-03-17 14:15:05.216,2021-03-17 14:25:15.469,10.170000000,2021-03-17
1,busmom1805@gmail.com,Walmart Tulsa,2021-03-17 05:40:46.530,2021-03-17 07:56:37.586,135.850000000,2021-03-17
2,renemora91@hotmail.com,Walmart Tulsa,2021-03-17 05:05:34.162,2021-03-17 06:26:25.224,80.850000000,2021-03-17
3,renemora91@hotmail.com,Walmart Tulsa,2021-03-17 10:39:49.862,2021-03-17 13:30:00.000,170.170000000,2021-03-17
4,renemora91@hotmail.com,Walmart Tulsa,2021-03-17 07:57:35.398,2021-03-17 09:30:00.000,92.400000000,2021-03-17


In [54]:
dw = pd.DataFrame(df.groupby(["MARKET_NM","DRVR_USER_ID","ACTV_DT"]).sum()).reset_index(level=[0,1])
dw.head()

C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_26700\2648669225.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.add is deprecated. In a future version, a TypeError will be raised. Before calling .add, select only columns which should be valid for the function.
  dw = pd.DataFrame(df.groupby(["MARKET_NM","DRVR_USER_ID","ACTV_DT"]).sum()).reset_index(level=[0,1])


,MARKET_NM,DRVR_USER_ID,ACTV_DUR_MIN_QTY
ACTV_DT,,,
2021-03-14,Walmart Orlando,1219paola@gmail.com,192.870000000
2021-04-21,Walmart Orlando,1219paola@gmail.com,180.050000000
2021-04-29,Walmart Orlando,1219paola@gmail.com,242.350000000
2021-05-03,Walmart Orlando,1219paola@gmail.com,51.680000000
2021-06-14,Walmart Orlando,1219paola@gmail.com,187.100000000


In [55]:
dw = dw.rename(columns={"DRVR_USER_ID": "driver", "ACTV_DUR_MIN_QTY": "minutes", "MARKET_NM": "market"})
dw

,market,driver,minutes
ACTV_DT,,,
2021-03-14,Walmart Orlando,1219paola@gmail.com,192.870000000
2021-04-21,Walmart Orlando,1219paola@gmail.com,180.050000000
2021-04-29,Walmart Orlando,1219paola@gmail.com,242.350000000
2021-05-03,Walmart Orlando,1219paola@gmail.com,51.680000000
2021-06-14,Walmart Orlando,1219paola@gmail.com,187.100000000
...,...,...,...
2022-07-14,Walmart Virginia Beach,zuzuhaf@gmail.com,754.920000000
2022-07-25,Walmart Virginia Beach,zuzuhaf@gmail.com,258.000000000
2022-07-26,Walmart Virginia Beach,zuzuhaf@gmail.com,890.120000000


In [56]:
dw['date'] = dw.index
dw['date'] = pd.to_datetime(dw['date'])
dw['dayofweek'] = dw['date'].dt.dayofweek

In [57]:
def weekday_check(dayofweek):
    
    if dayofweek in range(0,5):
        return "weekday"
    elif dayofweek in range(5,7):
        return "weekend"

In [58]:
dw["weekday-check"] = dw["dayofweek"].apply(lambda x: weekday_check(x))
dw

,market,driver,minutes,date,dayofweek,weekday-check
ACTV_DT,,,,,,
2021-03-14,Walmart Orlando,1219paola@gmail.com,192.870000000,2021-03-14,6,weekend
2021-04-21,Walmart Orlando,1219paola@gmail.com,180.050000000,2021-04-21,2,weekday
2021-04-29,Walmart Orlando,1219paola@gmail.com,242.350000000,2021-04-29,3,weekday
2021-05-03,Walmart Orlando,1219paola@gmail.com,51.680000000,2021-05-03,0,weekday
2021-06-14,Walmart Orlando,1219paola@gmail.com,187.100000000,2021-06-14,0,weekday
...,...,...,...,...,...,...
2022-07-14,Walmart Virginia Beach,zuzuhaf@gmail.com,754.920000000,2022-07-14,3,weekday
2022-07-25,Walmart Virginia Beach,zuzuhaf@gmail.com,258.000000000,2022-07-25,0,weekday
2022-07-26,Walmart Virginia Beach,zuzuhaf@gmail.com,890.120000000,2022-07-26,1,weekday


In [59]:
di = dict(dw["driver"].value_counts())

In [60]:
train = dw[dw["date"] < "2022-07-18"]
test = dw[dw["date"] >= "2022-07-18"]

In [62]:
train["freq"] = train["driver"].apply(lambda x: di[x])
train

C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_26700\2847001999.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["freq"] = train["driver"].apply(lambda x: di[x])


,market,driver,minutes,date,dayofweek,weekday-check,freq
ACTV_DT,,,,,,,
2021-03-14,Walmart Orlando,1219paola@gmail.com,192.870000000,2021-03-14,6,weekend,18
2021-04-21,Walmart Orlando,1219paola@gmail.com,180.050000000,2021-04-21,2,weekday,18
2021-04-29,Walmart Orlando,1219paola@gmail.com,242.350000000,2021-04-29,3,weekday,18
2021-05-03,Walmart Orlando,1219paola@gmail.com,51.680000000,2021-05-03,0,weekday,18
2021-06-14,Walmart Orlando,1219paola@gmail.com,187.100000000,2021-06-14,0,weekday,18
...,...,...,...,...,...,...,...
2022-07-11,Walmart Virginia Beach,zuzuhaf@gmail.com,898.180000000,2022-07-11,0,weekday,35
2022-07-12,Walmart Virginia Beach,zuzuhaf@gmail.com,754.220000000,2022-07-12,1,weekday,35
2022-07-13,Walmart Virginia Beach,zuzuhaf@gmail.com,901.240000000,2022-07-13,2,weekday,35


In [64]:
train = train.drop(['date','dayofweek'],axis=1)
train

,market,driver,minutes,weekday-check,freq
ACTV_DT,,,,,
2021-03-14,Walmart Orlando,1219paola@gmail.com,192.870000000,weekend,18
2021-04-21,Walmart Orlando,1219paola@gmail.com,180.050000000,weekday,18
2021-04-29,Walmart Orlando,1219paola@gmail.com,242.350000000,weekday,18
2021-05-03,Walmart Orlando,1219paola@gmail.com,51.680000000,weekday,18
2021-06-14,Walmart Orlando,1219paola@gmail.com,187.100000000,weekday,18
...,...,...,...,...,...
2022-07-11,Walmart Virginia Beach,zuzuhaf@gmail.com,898.180000000,weekday,35
2022-07-12,Walmart Virginia Beach,zuzuhaf@gmail.com,754.220000000,weekday,35
2022-07-13,Walmart Virginia Beach,zuzuhaf@gmail.com,901.240000000,weekday,35


In [66]:
dtrain = train.groupby(["market","driver","weekday-check","freq"]).mean().reset_index(level=[0,1])
dtrain

market                  driver     minutes
weekday-check freq                                                            
weekday       18           Walmart Orlando     1219paola@gmail.com  198.043846
weekend       18           Walmart Orlando     1219paola@gmail.com  255.070000
weekday       63           Walmart Orlando     24hrebels@gmail.com  277.248667
weekend       63           Walmart Orlando     24hrebels@gmail.com  359.760000
weekday       164          Walmart Orlando      69blesss@gmail.com  315.992072
...                                    ...                     ...         ...
              37    Walmart Virginia Beach  zouhair.bout@gmail.com   99.157407
weekend       37    Walmart Virginia Beach  zouhair.bout@gmail.com  103.941000
weekday       35    Walmart Virginia Beach       zuzuhaf@gmail.com  668.534583
weekend       35    Walmart Virginia Beach       zuzuhaf@gmail.com  438.932500
weekday       1     Walmart Virginia Beach      zyhariap@gmail.com   83.730000

[9335 rows x 3 columns]

In [67]:
dtrain.to_csv("Desktop/weekend-weekday_6markets-1.csv")

In [69]:
test["freq"] = test["driver"].apply(lambda x: di[x])
test = test.drop(['date','dayofweek'],axis=1)
test

C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_26700\492176521.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["freq"] = test["driver"].apply(lambda x: di[x])


,market,driver,minutes,weekday-check,freq
ACTV_DT,,,,,
2022-07-22,Walmart Orlando,24hrebels@gmail.com,57.990000000,weekday,63
2022-07-25,Walmart Orlando,24hrebels@gmail.com,122.520000000,weekday,63
2022-07-26,Walmart Orlando,24hrebels@gmail.com,436.360000000,weekday,63
2022-07-27,Walmart Orlando,24hrebels@gmail.com,232.330000000,weekday,63
2022-07-21,Walmart Orlando,aaliyahmarks06@gmail.com,446.190000000,weekday,54
...,...,...,...,...,...
2022-07-26,Walmart Virginia Beach,zoeeestaxx@icloud.com,492.950000000,weekday,21
2022-07-27,Walmart Virginia Beach,zoeeestaxx@icloud.com,310.140000000,weekday,21
2022-07-25,Walmart Virginia Beach,zuzuhaf@gmail.com,258.000000000,weekday,35


In [70]:
test = test.drop('freq',axis =1)
test

,market,driver,minutes,weekday-check
ACTV_DT,,,,
2022-07-22,Walmart Orlando,24hrebels@gmail.com,57.990000000,weekday
2022-07-25,Walmart Orlando,24hrebels@gmail.com,122.520000000,weekday
2022-07-26,Walmart Orlando,24hrebels@gmail.com,436.360000000,weekday
2022-07-27,Walmart Orlando,24hrebels@gmail.com,232.330000000,weekday
2022-07-21,Walmart Orlando,aaliyahmarks06@gmail.com,446.190000000,weekday
...,...,...,...,...
2022-07-26,Walmart Virginia Beach,zoeeestaxx@icloud.com,492.950000000,weekday
2022-07-27,Walmart Virginia Beach,zoeeestaxx@icloud.com,310.140000000,weekday
2022-07-25,Walmart Virginia Beach,zuzuhaf@gmail.com,258.000000000,weekday


In [71]:
test["date"] = test.index
test

,market,driver,minutes,weekday-check,date
ACTV_DT,,,,,
2022-07-22,Walmart Orlando,24hrebels@gmail.com,57.990000000,weekday,2022-07-22
2022-07-25,Walmart Orlando,24hrebels@gmail.com,122.520000000,weekday,2022-07-25
2022-07-26,Walmart Orlando,24hrebels@gmail.com,436.360000000,weekday,2022-07-26
2022-07-27,Walmart Orlando,24hrebels@gmail.com,232.330000000,weekday,2022-07-27
2022-07-21,Walmart Orlando,aaliyahmarks06@gmail.com,446.190000000,weekday,2022-07-21
...,...,...,...,...,...
2022-07-26,Walmart Virginia Beach,zoeeestaxx@icloud.com,492.950000000,weekday,2022-07-26
2022-07-27,Walmart Virginia Beach,zoeeestaxx@icloud.com,310.140000000,weekday,2022-07-27
2022-07-25,Walmart Virginia Beach,zuzuhaf@gmail.com,258.000000000,weekday,2022-07-25


In [72]:
test = test[test["date"] < "2022-07-25"]
test

,market,driver,minutes,weekday-check,date
ACTV_DT,,,,,
2022-07-22,Walmart Orlando,24hrebels@gmail.com,57.990000000,weekday,2022-07-22
2022-07-21,Walmart Orlando,aaliyahmarks06@gmail.com,446.190000000,weekday,2022-07-21
2022-07-20,Walmart Orlando,adrianafariasrocha@icloud.com,111.600000000,weekday,2022-07-20
2022-07-21,Walmart Orlando,adrianafariasrocha@icloud.com,273.230000000,weekday,2022-07-21
2022-07-22,Walmart Orlando,adrianafariasrocha@icloud.com,7.750000000,weekday,2022-07-22
...,...,...,...,...,...
2022-07-20,Walmart Virginia Beach,zoeeestaxx@icloud.com,145.150000000,weekday,2022-07-20
2022-07-21,Walmart Virginia Beach,zoeeestaxx@icloud.com,293.880000000,weekday,2022-07-21
2022-07-22,Walmart Virginia Beach,zoeeestaxx@icloud.com,93.470000000,weekday,2022-07-22


In [74]:
test = test.drop('date',axis=1)

In [76]:
dtest = test.groupby(["market","driver","weekday-check"]).mean().reset_index(level=[0,1])
dtest

,market,driver,minutes
weekday-check,,,
weekday,Walmart Orlando,24hrebels@gmail.com,57.990
weekday,Walmart Orlando,aaliyahmarks06@gmail.com,446.190
weekday,Walmart Orlando,adrianafariasrocha@icloud.com,130.860
weekend,Walmart Orlando,adrianafariasrocha@icloud.com,210.225
weekday,Walmart Orlando,adrianaleal80@yahoo.es,383.666
...,...,...,...
weekend,Walmart Virginia Beach,yodiancurry@gmail.com,551.870
weekday,Walmart Virginia Beach,zenmelika@gmail.com,246.818
weekend,Walmart Virginia Beach,zenmelika@gmail.com,192.570


In [77]:
dtest.to_csv('Desktop/testing.csv')

In [83]:
dtest['weekday-check'] = dtest.index

In [90]:
dtest = dtest.reset_index(drop=True)

In [93]:
final = pd.merge(dtrain,dtest,on=['driver','weekday-check'],how='left')

In [131]:
final

,market,weekday-check,driver,train_minutes,test_minutes
0,Walmart Orlando,weekday,1219paola@gmail.com,198.043846,0.00
1,Walmart Orlando,weekend,1219paola@gmail.com,255.070000,0.00
2,Walmart Orlando,weekday,24hrebels@gmail.com,277.248667,57.99
3,Walmart Orlando,weekend,24hrebels@gmail.com,359.760000,0.00
4,Walmart Orlando,weekday,69blesss@gmail.com,315.992072,0.00
...,...,...,...,...,...
9338,Walmart Virginia Beach,weekday,zouhair.bout@gmail.com,99.157407,0.00
9339,Walmart Virginia Beach,weekend,zouhair.bout@gmail.com,103.941000,0.00
9340,Walmart Virginia Beach,weekday,zuzuhaf@gmail.com,668.534583,0.00
9341,Walmart Virginia Beach,weekend,zuzuhaf@gmail.com,438.932500,0.00


In [95]:
final = final.drop(['market_y'],axis=1)

In [126]:
final = final.rename(columns = {"minutes_x":"train_minutes","minutes_y":"test_minutes","market_x":"market"})
final = final.fillna(0)

In [102]:
dtrain = dtrain.reset_index(drop=False)

In [121]:
dtrain = dtrain.set_index('weekday-check',drop=True)

In [135]:
final

,market,weekday-check,driver,train_minutes,test_minutes
0,Walmart Orlando,weekday,1219paola@gmail.com,198.043846,0.00
1,Walmart Orlando,weekend,1219paola@gmail.com,255.070000,0.00
2,Walmart Orlando,weekday,24hrebels@gmail.com,277.248667,57.99
3,Walmart Orlando,weekend,24hrebels@gmail.com,359.760000,0.00
4,Walmart Orlando,weekday,69blesss@gmail.com,315.992072,0.00
...,...,...,...,...,...
9338,Walmart Virginia Beach,weekday,zouhair.bout@gmail.com,99.157407,0.00
9339,Walmart Virginia Beach,weekend,zouhair.bout@gmail.com,103.941000,0.00
9340,Walmart Virginia Beach,weekday,zuzuhaf@gmail.com,668.534583,0.00
9341,Walmart Virginia Beach,weekend,zuzuhaf@gmail.com,438.932500,0.00
